In [ ]:
!pip3 install tqdm==4.29.1
!pip3 install shap
!pip3 install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 2.4 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling tqdm-4.64.1:
      Successfully uninstalled tqdm-4.64.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.4.2 requires tqdm<5.0.0,>=4.38.0, but you have tqdm 4.29.1 which is incompatible.
prophet 1.1.1 requires tqdm>=4.36.1, but you have tqdm 4.29.1 which is incompatible.
panel 0.12.1 requires tqdm>=4.48.0, but you have tqdm 4.29.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |███

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from scipy import stats
from sklearn.impute import SimpleImputer

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import OneClassSVM
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.metrics import mean_squared_error, precision_score, recall_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import plot_model
import lightgbm as lgb
import xgboost as xgb

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Dataset

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/BT4012 Project/Submission Folder/train_df.csv")
test_df = pd.read_csv("/content/drive/MyDrive/BT4012 Project/Submission Folder/test_df.csv")

In [ ]:
train_df.head()

,NPI,Tot_Benes_Mean,Tot_Srvcs_Mean,Tot_Bene_Day_Srvcs_Mean,Avg_Sbmtd_Chrg_Mean,Avg_Mdcr_Alowd_Amt_Mean,Avg_Mdcr_Pymt_Amt_Mean,Is_Fraud,Is_Male,Tot_Suplrs_Mean,...,Surgical Oncology,Thoracic Surgery,Undefined Physician type,Undersea and Hyperbaric Medicine,Urology,Vascular Surgery,Opioid_Tot_Drug_Cst_Missing,Opioid_Tot_Suply_Missing,Opioid_LA_Tot_Drug_Cst_Missing,Opioid_LA_Tot_Suply_Missing
0,1942294194,60.642857,96.642857,96.642857,216.477855,95.394587,74.656983,0.0,1.0,2.000000,...,0,0,0,0,0,0,0.0,0.0,1.0,1.0
1,1508965187,29.333333,32.833333,31.083333,463.666667,125.811621,99.238639,0.0,0.0,2.750000,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2,1992790315,28.000000,42.666667,42.666667,23.737179,7.180476,6.156374,0.0,1.0,9.000000,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
3,1538164637,177.828571,259.485714,245.971429,277.273075,90.160440,68.922831,0.0,1.0,3.666667,...,0,0,0,0,1,0,0.0,0.0,0.0,0.0
4,1336368661,43.653846,82.480769,65.807692,818.563984,261.450147,207.475945,0.0,1.0,1.600000,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0


In [ ]:
test_df.head()

,NPI,Tot_Benes_Mean,Tot_Srvcs_Mean,Tot_Bene_Day_Srvcs_Mean,Avg_Sbmtd_Chrg_Mean,Avg_Mdcr_Alowd_Amt_Mean,Avg_Mdcr_Pymt_Amt_Mean,Is_Fraud,Is_Male,Tot_Suplrs_Mean,...,Surgical Oncology,Thoracic Surgery,Undefined Physician type,Undersea and Hyperbaric Medicine,Urology,Vascular Surgery,Opioid_Tot_Drug_Cst_Missing,Opioid_Tot_Suply_Missing,Opioid_LA_Tot_Drug_Cst_Missing,Opioid_LA_Tot_Suply_Missing
0,1457324451,55.882353,83.000000,83.000000,120.765463,84.332661,74.155972,0.0,1.0,3.800000,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1,1588751523,83.478261,94.652174,93.913043,340.594074,170.506351,131.423367,0.0,1.0,15.000000,...,0,0,0,0,0,0,1.0,1.0,1.0,1.0
2,1437224128,61.666667,104.083333,100.500000,88.813892,45.756816,39.846102,0.0,1.0,12.500000,...,0,0,0,0,0,0,0.0,0.0,1.0,1.0
3,1689683864,24.142857,42.142857,42.142857,86.281071,68.183827,59.052302,0.0,1.0,5.666667,...,0,0,0,0,0,0,1.0,1.0,1.0,1.0
4,1215028956,124.742857,264.714286,245.800000,136.861692,96.403404,79.341149,0.0,1.0,9.000000,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0


In [ ]:
print(train_df["Is_Fraud"].value_counts())
print(test_df["Is_Fraud"].value_counts())

0.0    210665
1.0        85
Name: Is_Fraud, dtype: int64
0.0    52667
1.0       21
Name: Is_Fraud, dtype: int64


# Data Processing

In [ ]:
X_train, y_train = train_df.drop(columns=["NPI", "Is_Fraud"]), train_df["Is_Fraud"]
X_test, y_test = test_df.drop(columns=["NPI", "Is_Fraud"]), test_df["Is_Fraud"]

In [ ]:
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [ ]:
X_train.head()

,Tot_Benes_Mean,Tot_Srvcs_Mean,Tot_Bene_Day_Srvcs_Mean,Avg_Sbmtd_Chrg_Mean,Avg_Mdcr_Alowd_Amt_Mean,Avg_Mdcr_Pymt_Amt_Mean,Is_Male,Tot_Suplrs_Mean,Tot_Suplr_Benes_Mean,Tot_Suplr_Clms_Mean,...,Surgical Oncology,Thoracic Surgery,Undefined Physician type,Undersea and Hyperbaric Medicine,Urology,Vascular Surgery,Opioid_Tot_Drug_Cst_Missing,Opioid_Tot_Suply_Missing,Opioid_LA_Tot_Drug_Cst_Missing,Opioid_LA_Tot_Suply_Missing
0,-0.162963,-0.149922,-0.177403,-0.133186,0.046098,0.052374,0.814238,-0.578714,-0.487935,0.011593,...,-0.030197,-0.035078,-0.020205,-0.015558,-0.161204,-0.043443,-0.454475,-0.454475,1.365610,1.365610
1,-0.877559,-0.299360,-0.840665,0.480401,0.470132,0.484953,-1.228141,-0.362674,-0.487935,-0.384234,...,-0.030197,-0.035078,-0.020205,-0.015558,-0.161204,-0.043443,-0.454475,-0.454475,-0.732273,-0.732273
2,-0.907990,-0.276331,-0.723477,-0.611618,-1.183664,-1.153075,0.814238,1.437659,-0.014554,-0.087364,...,-0.030197,-0.035078,-0.020205,-0.015558,-0.161204,-0.043443,-0.454475,-0.454475,-0.732273,-0.732273
3,2.511633,0.231446,1.333345,0.017724,-0.026869,-0.048533,0.814238,-0.098625,-0.119750,0.105837,...,-0.030197,-0.035078,-0.020205,-0.015558,6.203312,-0.043443,-0.454475,-0.454475,-0.732273,-0.732273
4,-0.550713,-0.183089,-0.489360,1.361349,2.361022,2.389675,0.814238,-0.693936,-0.119750,-0.387061,...,-0.030197,-0.035078,-0.020205,-0.015558,-0.161204,-0.043443,-0.454475,-0.454475,-0.732273,-0.732273


In [ ]:
# under = RandomUnderSampler(sampling_strategy=0.1, random_state = 42)

# X_train_smote_under, y_train_smote_under = under.fit_resample(X_train.astype('float'), y_train)

# from collections import Counter
# print("Before SMOTE :" , Counter(y_train))
# print("After SMOTE :" , Counter(y_train_smote_under))

Before SMOTE : Counter({0.0: 210665, 1.0: 85})
After SMOTE : Counter({0.0: 850, 1.0: 85})


In [ ]:
# over = SMOTE(random_state = 42)

# X_train_smote, y_train_smote = over.fit_resample(X_train_smote_under.astype('float'), y_train_smote_under)

# from collections import Counter
# print("Before SMOTE :" , Counter(y_train))
# print("After SMOTE :" , Counter(y_train_smote))

Before SMOTE : Counter({0.0: 210665, 1.0: 85})
After SMOTE : Counter({0.0: 850, 1.0: 850})


In [ ]:
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours
smote_enn = SMOTEENN(sampling_strategy='minority',enn=EditedNearestNeighbours(sampling_strategy='all'), random_state=42)
X_train_smote_enn, y_train_smote_enn = smote_enn.fit_resample(X_train.astype('float'), y_train)
from collections import Counter
print("Before SMOTE :" , Counter(y_train))
print("After SMOTE :" , Counter(y_train_smote_enn))

Before SMOTE : Counter({0.0: 210665, 1.0: 85})
After SMOTE : Counter({1.0: 210665, 0.0: 209900})


In [ ]:
X_train_smote_enn.to_csv("/content/drive/MyDrive/BT4012 Project/Submission Folder/X_train_smote_enn.csv", index=False)
y_train_smote_enn.to_csv("/content/drive/MyDrive/BT4012 Project/Submission Folder/y_train_smote_enn.csv", index=False)